Forked from Jason Liu and RVK

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud,STOPWORDS
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
matplotlib.rcParams["figure.figsize"] = "8, 8"

In [ ]:
NewsComb = pd.read_csv('../input/Combined_News_DJIA.csv')

In [ ]:
NewsComb.head(5)

In [ ]:
NewsComb.shape

In [ ]:
NewsComb['Combined']=NewsComb.iloc[:,2:27].apply(lambda row: ''.join(str(row.values)),axis=1)

In [ ]:
non_down = NewsComb[NewsComb['Label']==1]
down = NewsComb[NewsComb['Label']==0]
print(len(non_down)/len(NewsComb))

In [ ]:
def to_words(content):
    letters = re.sub("[^a-zA-Z]"," ", content)
    words = letters.lower().split()
    stops = set(stopwords.words("english"))
    mwords = [w for w in words if not w in stops]
    return(" ".join( mwords))

In [ ]:
non_down_word = []
down_word = []
for word in non_down['Combined']:
    non_down_word.append(to_words(word))

for word in down['Combined']:
    down_word.append(to_words(word))

In [ ]:
down_word[0:5]

In [ ]:
wordcloud_down = WordCloud(background_color='black',
                          width=3000,
                          height=2500
                          ).generate(down_word[0])

In [ ]:
plt.figure(1,figsize=(8,8))
plt.imshow(wordcloud_down)
plt.axis('off')
plt.show()

In [ ]:
wordcloud_non = WordCloud(background_color='white',
                         width=3000,
                         height=2500
                         ).generate(non_down_word[0])

In [ ]:
plt.figure(1,figsize=(8,8))
plt.imshow(wordcloud_non)
plt.axis('off')
plt.show()

In [ ]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [ ]:
mix = []
for row in range(0,len(NewsComb.index)):
    mix.append(' '.join(str(x) for x in NewsComb.iloc[row,2:27]))

In [ ]:
mix[0:5]

In [ ]:
%time
tokenizer = RegexpTokenizer(r'\w+')
en_stop = get_stop_words('en')
ps = PorterStemmer()

texts = []

for item in mix:
    raw = item.lower()
    tokens = tokenizer.tokenize(raw)
    
    stopped_tokens = [item for item in tokens if not item in en_stop]
    stemmed_tokens = [ps.stem(item) for item in stopped_tokens]
    
    texts.append(stemmed_tokens)

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
%time
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary,passes=1,
                                     chunksize=10000,update_every=1)

In [ ]:
%time
import pyLDAvis.gensim

print(lda.print_topics(num_topics=10,num_words=3))

In [ ]:
lda.print_topics(5)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
mix_down = []
for row in range(0,len(down.index)):
    mix_down.append(' '.join(str(x) for x in down.iloc[row,2:27]))

In [ ]:
texts_down = []

for item in mix_down:
    raw = item.lower()
    tokens = tokenizer.tokenize(raw)
    
    stopped_tokens = [item for item in tokens if not item in en_stop]
    stemmed_tokens = [ps.stem(item) for item in stopped_tokens]
    
    texts_down.append(stemmed_tokens)

dictionary_down = corpora.Dictionary(texts_down)
corpus_down = [dictionary_down.doc2bow(text) for text in texts_down]

In [ ]:
lda_down = gensim.models.ldamodel.LdaModel(corpus_down, num_topics=10, id2word=dictionary_down,passes=1,
                                     chunksize=10000,update_every=1)

In [ ]:
print(lda_down.print_topics(num_topics=10,num_words=3))

In [ ]:
lda_down.print_topics(5)

In [ ]:
pyLDAvis.gensim.prepare(lda_down, corpus_down, dictionary_down)

In [ ]:
mix_non_down = []
for row in range(0,len(non_down.index)):
    mix_non_down.append(' '.join(str(x) for x in non_down.iloc[row,2:27]))

In [ ]:
texts_non_down = []

for item in mix_non_down:
    raw = item.lower()
    tokens = tokenizer.tokenize(raw)
    
    stopped_tokens = [item for item in tokens if not item in en_stop]
    stemmed_tokens = [ps.stem(item) for item in stopped_tokens]
    
    texts_non_down.append(stemmed_tokens)

dictionary_non_down = corpora.Dictionary(texts_non_down)
corpus_non_down = [dictionary_non_down.doc2bow(text) for text in texts_non_down]

In [ ]:
lda_non_down = gensim.models.ldamodel.LdaModel(corpus_non_down, num_topics=10, id2word=dictionary_non_down,
                                               passes=1, chunksize=10000,update_every=1)

In [ ]:
print(lda_non_down.print_topics(num_topics=10,num_words=3))

In [ ]:
lda_non_down.print_topics(5)

In [ ]:
pyLDAvis.gensim.prepare(lda_non_down, corpus_non_down, dictionary_non_down)